**Configuración**

In [1]:

COINS = ["BTC", "ETH", "ADA"]         # monedas a descargar
FIAT  = "USD"                         # moneda de referencia

START_DATE = "2024-01-01"            
END_DATE   = "2024-03-31"             

OUTPUT_CSV = "data/crypto_limpio.csv"

import os
os.makedirs("data", exist_ok=True)

COINS, FIAT, START_DATE, END_DATE, OUTPUT_CSV

(['BTC', 'ETH', 'ADA'],
 'USD',
 '2024-01-01',
 '2024-03-31',
 'data/crypto_limpio.csv')

monedas de referencia y a descargar: bitcoin, ethereum y cardano. Se especifican las fechas de las cuales se extraen los datos y la ruta donde queda el archivo csv ya limpio. 

**cargar API Key de CryptoCompare**

In [2]:
import os
try:
    from dotenv import load_dotenv
    load_dotenv()
except Exception:
    pass

API_KEY_CC = os.getenv("CRYPTOCOMPARE_KEY", None)
print("CryptoCompare key:", "sí (oculta)" if API_KEY_CC else "no (modo público)")


CryptoCompare key: sí (oculta)


Se debe tener un archivo .env para allí meter la API Key
¿Para que una API Key? Se utilizó para una mejor extracción de datos en CryptoCompare. Se crea una cuenta y la API Key la brindan muy facil. Es importante tambien por la autenticacion y autorizacion asi como el monitoreo de uso y extraccion. 

**descarga desde cryptocompare** 

In [3]:
import time, requests
from datetime import datetime, timedelta
from dateutil import tz

BASE_URL_CC = "https://min-api.cryptocompare.com/data/v2/histoday"
MAX_LIMIT = 2000  # máximo de días por request

def fetch_histoday_cc(fsym: str, tsym: str, start_date: str, end_date: str, api_key: str|None):
    start = datetime.strptime(start_date, "%Y-%m-%d").date()
    end   = datetime.strptime(end_date,   "%Y-%m-%d").date()
    frames = []
    to_dt = datetime.combine(end, datetime.min.time())
    while True:
        limit = min(MAX_LIMIT, (to_dt.date() - start).days)
        params = {"fsym": fsym, "tsym": tsym, "limit": limit, "toTs": int(to_dt.replace(tzinfo=tz.UTC).timestamp())}
        headers = {"authorization": f"Apikey {api_key}"} if api_key else {}
        r = requests.get(BASE_URL_CC, params=params, headers=headers, timeout=60)
        r.raise_for_status()
        js = r.json()
        if js.get("Response") != "Success":
            raise RuntimeError(js.get("Message"))
        df = pd.DataFrame(js["Data"]["Data"])
        if df.empty:
            break
        df["time"] = pd.to_datetime(df["time"], unit="s").dt.date
        df = df[(df["time"]>=start) & (df["time"]<=end)]
        frames.append(df)
        if df["time"].min() <= start:
            break
        to_dt = datetime.combine(df["time"].min(), datetime.min.time()) - timedelta(days=1)
        time.sleep(0.6)
    if not frames:
        return pd.DataFrame(columns=["time","open","high","low","close","volumeto"])
    out = pd.concat(frames, ignore_index=True).sort_values("time")
    out = out[["time","open","high","low","close","volumeto"]]  # nos quedamos con lo necesario
    return out



Descarga por dias (Histoday) y devuelve columnas crudas; luego se limpia

**Se definen reglas de limpieza estricta**

In [4]:
import pandas as pd
import numpy as np

def _numeric(df, cols):
    for c in cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

def _drop_invalid_ohlc_rows(df):
    req = ["time","open","high","low","close","volumeto"]
    df = df.dropna(subset=req)
    df = df[(df["open"]>0) & (df["high"]>0) & (df["low"]>0) & (df["close"]>0)]
    df = df[df["volumeto"]>=0]
    df = df[df["high"]>=df["low"]]
    df = df[(df["open"].between(df["low"], df["high"])) & (df["close"].between(df["low"], df["high"]))]
    return df

def clean_frame_strict(df: pd.DataFrame, coin_symbol: str) -> pd.DataFrame:
    if df is None or df.empty:
        return pd.DataFrame(columns=["time","open","high","low","close","volumeto","coin"])
    df = df.drop_duplicates(subset=["time"]).copy()
    df = _numeric(df, ["open","high","low","close","volumeto"])
    for c in ["open","high","low","close"]:
        df.loc[df[c] <= 0, c] = np.nan
    df.loc[df["volumeto"] < 0, "volumeto"] = np.nan
    df = _drop_invalid_ohlc_rows(df)
    df["coin"] = coin_symbol.lower()
    df = df[["time","open","high","low","close","volumeto","coin"]].reset_index(drop=True)

    # validaciones fuertes
    assert not df.isna().any().any()
    assert not df.duplicated(subset=["coin","time"]).any()
    return df



Deja el dataset 100% limpio: sin nulos, sin duplicados coin+time, sin ceros o negativos, OHCL (Open, High, Low, Close) de manera consistente 

**coin+time** = clave compuesta que se usa para identificar de forma unica cada fila del dataset
coin = moneda
time = le fecha del registro
juntas forman una llave unica: no puede existir mas de un registro con la misma moneda y la misma fecha.
Ejemplo: (coin='btc', time='2024-02-15') identifica exactamente la vela/registro de BTC del 15-02-2024.

**histoday** 
Endpoint de la API de CryptoCompare que devuelve velas diarias (historical to day)
cada fila representa un dia con sus valores OHCL y volumen
Se usó porque entrega justo lo que se necesita para el sprint: serie diaria por moneda en un rango de fechas


**API 2 sin API KEY BINANCE**

In [ ]:

import requests
from datetime import datetime, timezone

def _to_ms(date_str: str) -> int:
    return int(datetime.strptime(date_str, "%Y-%m-%d").replace(tzinfo=timezone.utc).timestamp() * 1000)

def fetch_binance_daily(symbol_pair: str, start_date: str, end_date: str) -> pd.DataFrame:
    """
    Ejemplo symbol_pair: 'BTCUSDT', 'ETHUSDT', 'ADAUSDT'
    Devuelve columnas: time, open, high, low, close, volumeto
    - volumen es el 'volume' de Binance (cantidad del activo base).
    """
    url = "https://api.binance.com/api/v3/klines"
    params = {
        "symbol": symbol_pair,
        "interval": "1d",
        "startTime": _to_ms(start_date),
        "endTime": _to_ms(end_date),
        "limit": 1000
    }
    r = requests.get(url, params=params, timeout=60)
    r.raise_for_status()
    data = r.json()
    if not data:
        return pd.DataFrame(columns=["time","open","high","low","close","volumeto"])

    #
    rows = []
    for d in data:
        rows.append({
            "time":      pd.to_datetime(d[0], unit="ms").date(),
            "open":      float(d[1]),
            "high":      float(d[2]),
            "low":       float(d[3]),
            "close":     float(d[4]),
            "volumeto":  float(d[5]),   
        })
    df = pd.DataFrame(rows).sort_values("time")
    return df[["time","open","high","low","close","volumeto"]]



Coingecko entrega listas por milisegundos. Construimos close y, si falta OHCL se aproxima con close, la limpieza elimina dias raros

**Asignacion de fuente por moneda**

In [14]:
SOURCE_BY_COIN = {
    "BTC": "cc",  # CryptoCompare
    "ETH": "bn",  # Binance  
    "ADA": "cc",  
}
SOURCE_BY_COIN

{'BTC': 'cc', 'ETH': 'bn', 'ADA': 'cc'}

Como tenemos dos API para extraccion de datos, CryptoCompare y Binance elegimos que datos de que moneda extraemos de la API que queremos. 
SOURCE_BY_COIN la almacena. De esta manera es que tenemos total libertad de elegir entre dos API. 


**Descarga por moneda segun fuente, limpieza y unión**

In [ ]:

OUTPUT_CSV = "data/crypto_limpio.csv"
all_frames = []

for sym in COINS:
    src = SOURCE_BY_COIN.get(sym, "cc")
    print(f"> {sym}: usando {src.upper()} ...")

    if src == "cc":
        raw = fetch_histoday_cc(sym, FIAT, START_DATE, END_DATE, API_KEY_CC)
    elif src == "bn":
        pair = f"{sym}USDT"  
        raw = fetch_binance_daily(pair, START_DATE, END_DATE)
    else:
        raise ValueError("Fuente desconocida (usa 'cc' o 'bn')")

    print(f"   crudo: {len(raw)} filas")
    clean = clean_frame_strict(raw, sym)
    print(f"   limpio: {len(clean)} filas")
    all_frames.append(clean)

# unir todo y ordenar
final_df = pd.concat(all_frames, ignore_index=True).sort_values(["coin","time"])

# validaciones sencillas
assert not final_df.isna().any().any(), "Hay NaN en el dataset final."
assert not final_df.duplicated(subset=["coin","time"]).any(), "Hay duplicados coin+time."
assert final_df[["open","high","low","close"]].gt(0).all().all(), "OHLC con ceros/negativos."
assert (final_df["volumeto"] >= 0).all(), "Volumen negativo."
assert (final_df["high"] >= final_df["low"]).all(), "high < low."
assert final_df["open"].between(final_df["low"], final_df["high"]).all(), "open fuera de rango."
assert final_df["close"].between(final_df["low"], final_df["high"]).all(), "close fuera de rango."

# Guardar CSV final 
final_df.to_csv(OUTPUT_CSV, index=False)
print(f"\nCSV guardado: {OUTPUT_CSV} | shape: {final_df.shape}")

# Vista rápida 
display(final_df.head(10))
print("\nCobertura por moneda:")
display(final_df.groupby('coin')['time'].agg(['min','max','count']))





> BTC: usando CC ...
   crudo: 91 filas
   limpio: 91 filas
> ETH: usando BN ...
   crudo: 91 filas
   limpio: 91 filas
> ADA: usando CC ...
   crudo: 91 filas
   limpio: 91 filas

CSV guardado: data/crypto_limpio.csv | shape: (273, 7)


,time,open,high,low,close,volumeto,coin
182,2024-01-01,0.5938,0.6242,0.5905,0.6234,19115404.63,ada
183,2024-01-02,0.6234,0.6372,0.6022,0.6052,29680628.54,ada
184,2024-01-03,0.6052,0.6187,0.5346,0.5572,53874007.80,ada
185,2024-01-04,0.5572,0.5790,0.5499,0.5704,27363954.90,ada
186,2024-01-05,0.5704,0.5731,0.5226,0.5419,31754702.64,ada
187,2024-01-06,0.5419,0.5433,0.5075,0.5231,20137921.97,ada
188,2024-01-07,0.5231,0.5303,0.4883,0.4946,20255458.38,ada
189,2024-01-08,0.4946,0.5448,0.4650,0.5416,42975948.45,ada
190,2024-01-09,0.5416,0.5438,0.4966,0.5123,30498183.66,ada
191,2024-01-10,0.5123,0.5897,0.4879,0.5662,48877074.88,ada



Cobertura por moneda:


,min,max,count
coin,,,
ada,2024-01-01,2024-03-31,91
btc,2024-01-01,2024-03-31,91
eth,2024-01-01,2024-03-31,91
